In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

#import necessary libraries.
import tensorflow as tf
import numpy as np
layer = tf.keras.layers

print('check tensorflow version : ', tf.__version__)

# tf.data API

tf.data API는 데이터셋을 모델에 연결해주기 위한 복합적인 입력 파이프라인을 구축할 수 있게 도와줍니다.

다수의 분산된 파일로부터 통합된 데이터를 만들어야 하는 경우나, 데이터 전처리, 미니배치, 랜덤셔플링 등의 데이터 파이프라인을 위한 복잡한 구조를 높은 추상성으로 간단하게 제어할 수 있는 인터페이스를 제공합니다.

https://www.tensorflow.org/guide/data

### 1. from_tensor_slices : python으로부터 데이터 받기

In [ ]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))

dataset1.element_spec

In [ ]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))

dataset2.element_spec

In [ ]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3.element_spec

In [ ]:
for i in dataset3.take(1):
    print(i)
#     print(i[0])
#     print('\n-------------------------------------------------------------------\n')
#     print(i[1])

메모리 공간 위 이미지를 가져오는 경우

In [ ]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
images, labels = train
images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

### 2. TextLineDataset : text file로부터 데이터셋 생성

In [ ]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url+file_name)
    for file_name in file_names ]

In [ ]:
print(file_paths)

In [ ]:
dataset = tf.data.TextLineDataset(file_paths)

In [ ]:
for line in dataset.take(1):
  print(line.numpy())

In [ ]:
new_dataset = dataset.shuffle(buffer_size=10000).batch(3)

In [ ]:
for line in new_dataset.take(1):
    print(line.numpy())

### 3. TextLineDataset : csv 파일로부터 읽어오기

In [ ]:
import pandas as pd
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
df = pd.read_csv(titanic_file, index_col=None)
df.head()

In [ ]:
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

# keras model build

keras에서 모델을 만들기 위해 알아야 할 3가지 자료형인 layer, tensor, model이 있다.
layer는 인공신경망을 구성하는 하위 계층을 의미하고, tensor는 layer와 layer 사이의 입력과 출력을 말한다. TensorFlow는 flow graph의 형태이기 때문에 tensor는 데이터에 따라 계속 변하는 변동 변수이기 때문에 특정 값을 가진다고 할 수 없음. 그러나 tf 2.0부터 tensor의 값을 tensor.numpy()의 형태로 쉽게 확인할 수 있음. 마지막으로 model은 layer를 엮은 네트워크 객체라고 할 수 있다. 

In [ ]:
#layer는 tf.keras.layers 아래의 클래스로 만들 수 있다.
d1 = tf.keras.layers.Dense(32, activation='relu')

In [ ]:
print(type(d1)) #레이어의 타입을 볼 수 있다.

In [ ]:
# 각 layer 클래스는 method로 init, build, call를 갖는다. 
# init은 객체가 만들어지는 단계이고, tf.keras.layers.Dense(32, activation='relu')를 선언했을 때 실행된다.
# build는 실제로 layer가 파라미터를 갖는 단계, call은 layer가 가진 parameter로 데이터를 계산하는 단계이다.

print(d1.get_weights()) #weight는 build되지 않았기 때문에 없음.

In [ ]:
# build는 layer가 첫 번째로 call됐을 때 파라미터를 생성함. 특정 데이터를 입력하기 애매하기 때문에 보통 tf.keras.Input을 사용함.
inputs = tf.keras.Input(shape=(784)) #shape를 정해줌. 
#layer의 input shape이 정해져야 layer가 build 단계에서 파라미터의 shape을 결정할 수 있음.
print('type of inputs', type(inputs))
d1_output = d1(inputs)
print('weights of d1', d1.get_weights(), 
      '\n-------------------------------------------------\n',
      'weight shape : ', d1.get_weights()[0].shape,
      '\n-------------------------------------------------\n',
      'bias shape : ', d1.get_weights()[1].shape)

In [ ]:
# build가 된 layer는 이제 입력으로 데이터를 받으면 output을 계산한다.
d1_output = d1(np.ones([1,784], dtype=np.float32))
print('d1_output의 타입', type(d1_output)) # 계산 결과의 자료형은 tensor
print('\n-------------------------------------------------\n')
print(d1_output.numpy()) #tf 2.0은 eager excution을 default로 제공하여 현재 tensor가 가지고 있는 값을 확인할 수 있음.

In [ ]:
#model은 다음과 같이 첫 번째 layer의 input tensor와 마지막 layer의 output tensor를 입력 받아 생성한다.

inputs = tf.keras.Input(shape=(64), name='data_input')
d_1 = tf.keras.layers.Dense(32, activation='relu')
d_1_output = d_1(inputs)
# d_1 = tf.keras.layers.Dense(32, activation='relu')(inputs)와 같이 init 후 inputs를 넣어서 build를 동시에 수행할 수 있음.
d_2_output = tf.keras.layers.Dense(64, activation='relu')(d_1_output)

model = tf.keras.Model(inputs=inputs, outputs=d_2_output)

In [ ]:
model.summary()

In [ ]:
#make sure you have already graphviz, pydot, pydotplus libraries.
tf.keras.utils.plot_model(model, 'my_first_model_with_shape_info.png', show_shapes=True)

## 1. Sequential model

### data pipeline creation

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)

In [ ]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [ ]:
print("The shape of train dataset : ", x_train.shape)
print("The shape of test dataset : ", x_test.shape)

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32).repeat()

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

# Trains for 5 epochs.
import math
steps_per_epoch=math.ceil(60000/32)
model.fit(train_ds, epochs=5, steps_per_epoch=steps_per_epoch)
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.

In [ ]:
model.evaluate(test_ds)

In [ ]:
del model
tf.keras.backend.clear_session()

## 2. Functional API

In [ ]:
inputs = tf.keras.Input(shape=(28,28,1))  # Returns a placeholder tensor
x = tf.keras.layers.Flatten()(inputs)
# A layer instance is callable on a tensor, and returns a tensor.
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
predictions = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [ ]:
next(iter(train_ds))[1]

In [ ]:
# The compile step specifies the training configuration.
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

# Trains for 5 epochs.
import math
steps_per_epoch=math.ceil(60000/32)
model.fit(train_ds, epochs=5, steps_per_epoch=steps_per_epoch)
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.

In [ ]:
model.evaluate(test_ds)

In [ ]:
del model
tf.keras.backend.clear_session()

# Split pre-trained model for customized transfer learning

In [ ]:
# You can take famous image processing architecture, resnet101.


#https://keras.io/applications/#usage-examples-for-image-classification-models

'''
include_top: whether to include the fully-connected layer at the top of the network.
weights: one of None (random initialization) or 'imagenet' (pre-training on ImageNet).
input_tensor: optional Keras tensor (i.e. output of layers.Input()) to use as image input for the model.
input_shape: optional shape tuple, only to be specified if include_top is False (otherwise the input shape has to be (299, 299, 3). It should have exactly 3 inputs channels, and width and height should be no smaller than 71. E.g. (150, 150, 3) would be one valid value.
pooling: Optional pooling mode for feature extraction when include_top is False.
None means that the output of the model will be the 4D tensor output of the last convolutional layer.
'avg' means that global average pooling will be applied to the output of the last convolutional layer, and thus the output of the model will be a 2D tensor.
'max' means that global max pooling will be applied.
classes: optional number of classes to classify images into, only to be specified if include_top is  True, and if no weights argument is specified.
'''


inputs = tf.keras.Input(shape=(240, 240, 3))
resnet101 = tf.keras.applications.ResNet101(include_top=False, weights='imagenet', input_tensor=inputs)

In [ ]:
for l in resnet101.layers[:10]:
  print(l)

In [ ]:
#plot the architecture of resnet 101
tf.keras.utils.plot_model(resnet101, 'resnet101.png', show_shapes=True)

In [ ]:
model_input = resnet101.get_layer('input_5').input

In [ ]:
model_output = resnet101.get_layer('conv4_block23_out').output

In [ ]:
new_model = tf.keras.Model(inputs=model_input, outputs=model_output)

In [ ]:
new_model.summary()